In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,8)

# # 青海省部分地区
roi = ee.Geometry.Rectangle([89.8565, 34.6503,91.4664, 35.8282])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    water_area = image.select('mNDWI').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })
    waterareas = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.setMulti({'imagearea': imageareas.get('B2'),'waterarea': waterareas.get('mNDWI')})

In [5]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic(),visParams,'landsat image')
print(landsat_images.size().getInfo())

134


# 筛选得出采样影像

In [6]:
image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
# print(areas.getInfo())

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('imagearea',ee.Number(areas.get('B2')).divide(7)))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

39
['2018-01-11', '2018-02-12', '2018-03-16', '2018-04-01', '2018-04-17', '2018-05-03', '2018-06-04', '2018-07-22', '2018-01-02', '2018-01-18', '2018-02-03', '2018-03-07', '2018-03-23', '2018-04-08', '2018-04-24', '2018-05-10', '2018-05-26', '2018-06-27', '2018-08-30', '2018-11-02', '2018-11-18', '2018-12-04', '2018-01-18', '2018-02-03', '2018-03-07', '2018-03-23', '2018-04-08', '2018-04-24', '2018-05-10', '2018-05-26', '2018-06-27', '2018-07-13', '2018-07-29', '2018-08-30', '2018-10-01', '2018-11-02', '2018-11-18', '2018-12-04', '2018-12-20']


In [50]:
# 需要校正的图像
corrected_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('waterarea',5))
print(corrected_image.aggregate_array('waterarea').size().getInfo())
print(corrected_image.aggregate_array('waterarea').getInfo())

# 不需要校正的图像
No_corrected_image = landsat_images.map(area_after_clip).filter(ee.Filter.lte('waterarea',5))
print(No_corrected_image.aggregate_array('waterarea').size().getInfo())
print(No_corrected_image.aggregate_array('waterarea').getInfo())

118
[66.0155230625, 79.26884221960789, 10.002199875, 73.96197908676476, 25.255907749264704, 484.2757035181371, 26.0041435625, 55.974048067647054, 71.01584468749999, 41.00601456249999, 229.03525568749998, 62.14305773039216, 229.03409168749985, 66.75894824877453, 86.01535375000002, 446.87805984019576, 365.5723914348036, 225.43155898725485, 183.33483238602946, 25.59652792941176, 30.870329047794122, 116.50294937867642, 5.001415625000001, 57.00295881250001, 23.103395278921575, 46.00477281249998, 23.03795008235294, 58.00898475, 152.60321806715692, 48.377746238480384, 53.963216855147046, 154.085629540196, 24.003983062499998, 181.98232706250008, 193.0266611875002, 84.00616793750001, 111.36567825269607, 349.2591277612748, 609.5249387897061, 243.19701445098016, 101.07009715563727, 114.87406599705875, 387.7724492340684, 132.45155612524502, 375.3203504348032, 105.82368987254904, 1185.0735567102988, 176.97727656985302, 301.0559131764705, 663.8491605504898, 962.3392269833338, 416.06174908701, 25.312

# 样本采集函数的定义

In [8]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# Map.addLayer(index_waterfrequency.gte(0.75).selfMask(),{'palette':['green']},"index permanentwater")
# JRC
JRC_2018 = ee.Image("JRC/GSW1_3/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# # Maryland
# Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(roi).select('b1').rename('waterclass')
# Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
# Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
# Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# # 三者的交集
# permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
# landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
# seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
# 两者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

In [17]:
def intervalmean(image):
    awei = image.updateMask(seasonWaterExtent).select('AWEI')
    intervalMean = awei.reduceRegion(**{
        'reducer': ee.Reducer.intervalMean(50,100), 
        'geometry': image.geometry(), 
        'scale': 30, 
        'bestEffort': True
    })
    return image.set({'mean': intervalMean.get('AWEI')})

In [23]:
a = sample_image.limit(39).map(intervalmean)
b = a.aggregate_array('mean')
print(b.getInfo())

[19.356971760986816, -85.64451092048252, 359.5121798913315, -866.7328699272574, 540.1160331012488, 1777.7738804960036, 1007.2344912479166, 1152.5850265137228, 414.3526731422403, 335.4687853676186, 66.64383904707908, 631.6947253117634, -222.31249778945045, 800.2620113431273, 1950.3032687582831, 1729.1422850711137, 1778.9482477531044, 1383.0189348182882, 1797.4908362794483, 2234.0405087318204, 3114.7011182576493, 1317.8155994719834, 462.696912057167, 381.5587216722168, 288.4563600541863, 275.34245818982674, 2619.084642123285, 2876.1390043130104, 1248.434905510485, 875.2651914121343, 2272.179513736374, 1256.4834062364155, 1614.1603262380827, 2371.2147192845164, 2281.0353420958604, 2508.4323144418145, 4874.229760478297, 2298.634409983928, 1626.8956009328258]


In [24]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI']
# 分类标签
label = 'waterclass'
def imageSample(image):
    ROI = image.geometry()
    permanent = permanentWaterExtent.updateMask(image.select('B3').gt(0)).remap([0,1],[0,2]).rename('waterclass')
    land = landExtent.updateMask(image.select('B3').gt(0))
    awei = image.updateMask(seasonWaterExtent).select('AWEI')
    season_class = awei.gt(ee.Image.constant(image.get('mean'))).rename('waterclass')
    seasonalwater_Extent = ee.ImageCollection([season_class,basemap]).sum().remap([0,1],[0,2]).rename('waterclass')
    percentile_waterclass = ee.ImageCollection([permanent,seasonalwater_Extent,land]).sum().remap([0,1,2],[0,0,1]).rename('waterclass')
    water = percentile_waterclass.selfMask()
    nowater = percentile_waterclass.eq(0).selfMask().remap([1],[0]).rename('waterclass')
    water_points = percentile_waterclass.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 200,
        'seed': 1,
        'geometries': True,
    })
    nowater_points = nowater.sample(**{
        'region': ROI,
        'scale': 30,
        'numPixels': 50,
        'seed': 1,
        'geometries': True,
    })
    all_points = water_points
    #.merge(nowater_points)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30,
        'geometries': True,
    })
    return sample_points

In [28]:
# 总的样本点数
points_collection = a.map(imageSample).flatten()
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))

# # 水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# # 非水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [91.21998487294526, 35.27136878677897]}, 'id': 'LC08_138036_20180111_2_0', 'properties': {'AWEI': -2405, 'B2': 647, 'B3': 932, 'B4': 1176, 'B5': 1422, 'B6': 1883, 'B7': 1698, 'NDVI': 0.09468822181224823, 'mNDWI': -0.337833046913147, 'ndvi_mndwi': 0.432521253824234, 'waterclass': 0}}
total sample number:2199


In [68]:
label = 'qinghai_2018'

# csv格式
selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI','ndvi_mndwi','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder='wuhan', file_format='csv', selectors=selector)

# shp格式
selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI','ndvi_mndwi','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder='wuhan', file_format='shp', selectors=selector)

Exporting qinghai_2018...
Exporting qinghai_2018...


In [69]:
def AutomaticCorrection(image):
    basemap = ee.Image.constant(0).toFloat().clip(image.geometry()).clip(roi).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence').clip(image.geometry()).clip(roi)
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': image.clip(roi).geometry(),
        'scale': 30,
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    # print(occurrence_threshold.getInfo())
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

In [70]:
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

#对影像进行分类
def training(image):
    return image.select(bands).classify(trainedClassifier).eq(1).rename('waterclass')

In [71]:
waterpixel_image = corrected_image.map(training)

# print(waterpixel_image.size().getInfo())

AttributeError: 'Image' object has no attribute 'map'

In [ ]:
correct_image = AutomaticCorrection(waterpixel_image.first())

In [ ]:
# 需要校正的图像
correct_image = waterpixel_image.map(AutomaticCorrection)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
raw_image = maskL8sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_139036_20180627').clip(roi))
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(landsat_images.mosaic().clip(raw_image.geometry()), region=region, vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'reference image', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(enhanced_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'classified image(AutomaticCorrection)', fontsize=20)